In [2]:
import torch
import torchvision
import cv2
import numpy as np
from pathlib import Path
from boxmot import BotSort
from torchvision.models.detection import FasterRCNN_ResNet50_FPN_Weights

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [4]:
detector = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights = FasterRCNN_ResNet50_FPN_Weights.COCO_V1)
detector.eval().to(device)

# Initialize the tracker
tracker = BotSort(
    reid_weights=Path('osnet_x0_25_msmt17.pt'),  # Path to ReID model
    device=device,  # Use CPU for inference
    half=False
)

2024-10-10 20:33:02.100 | INFO     | boxmot.utils.torch_utils:select_device:52 - Yolo Tracking v11.0.2 🚀 Python-3.10.6 torch-2.2.1+cu118
CUDA:0 (NVIDIA GeForce RTX 4070 SUPER, 12282MiB)
2024-10-10 20:33:02.125 | SUCCESS  | boxmot.appearance.reid_model_factory:load_pretrained_weights:183 - Loaded pretrained weights from osnet_x0_25_msmt17.pt


In [5]:
vid = cv2.VideoCapture('D:\\041.낙상사고 위험동작 영상-센서 쌍 데이터\\3.개방데이터\\1.데이터\\Training\\01.원천데이터\\TS (2)\\영상\\Y\\BY\\00050_H_A_BY_C1.mp4')  # or 'path/to/your.avi
if not vid.isOpened() :
    print('못열었음')
    exit()

fps = vid.get(cv2.CAP_PROP_FPS)
frame_w = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_h = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))

total_frame = fps * 10

result_path = 'ehlfRK.mp4' 
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(result_path, fourcc, 25, (frame_w, frame_h))
frame_count = 0

while True:
    # Capture frame-by-frame
    ret, frame = vid.read()
    if not ret or frame_count >= total_frame : 
        break
    # If ret is False, it means we have reached the end of the video
    if not ret:
        break

    # Convert frame to tensor and move to device
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_tensor = torchvision.transforms.functional.to_tensor(frame).to(device)

    # Perform detection
    with torch.no_grad():
        detections = detector([frame_tensor])[0]

    # Filter the detections (e.g., based on confidence threshold)
    confidence_threshold = 0.5
    dets = []
    for i, score in enumerate(detections['scores']):
        if score >= confidence_threshold:
            bbox = detections['boxes'][i].cpu().numpy()
            label = detections['labels'][i].item()
            conf = score.item()
            dets.append([*bbox, conf, label])

    # Convert detections to numpy array (N X (x, y, x, y, conf, cls))
    dets = np.array(dets)

    # Update the tracker
    res = tracker.update(dets, frame)  # --> M X (x, y, x, y, id, conf, cls, ind)

    # Plot tracking results on the image
    tracker.plot_results(frame, show_trajectories=True)
    out.write(frame)
    
    cv2.imshow('BoXMOT + Torchvision', frame)
    
    frame_count += 1
    # Simulate wait for key press to continue, press 'q' to exit
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

# Release resources
vid.release()
out.release()
cv2.destroyAllWindows()

못열었음


: 